In [16]:
import numpy as np
import damask
import shutil
import os
from math import * 
os.getcwd()
print("The current directory is: ")
print(os.getcwd())
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
print("Moving up to the parent directory")
print(os.getcwd())

In [2]:
loading_paths = [
    "PH_linear_uniaxial_RD",
    "PH_linear_uniaxial_TD",
    "PH_nonlinear_biaxial_RD",
    "PH_nonlinear_biaxial_TD",
    "PH_nonlinear_planestrain_RD",
    "PH_nonlinear_planestrain_TD",
    "PH_nonlinear_uniaxial_RD",
    "PH_nonlinear_uniaxial_TD",
    "DB_linear_uniaxial_RD",
    "DB_linear_uniaxial_TD",
    "DB_nonlinear_biaxial_RD",
    "DB_nonlinear_biaxial_TD",
    "DB_nonlinear_planestrain_RD",
    "DB_nonlinear_planestrain_TD",
    "DB_nonlinear_uniaxial_RD",
    "DB_nonlinear_uniaxial_TD",
]

In [18]:
for loading in loading_paths:
    src_file = 'damask3/damask3_template/preprocessing.sh'
    dst_file = f'damask3/templates/{loading}'
    shutil.copy2(src_file, dst_file)

In [3]:
numerics = damask.Config(f"""
grid:
  itmin: 2
  itmax: 250
  maxCutBack: 5
""")

for loading in loading_paths:
    numerics.save(f"damask3/templates/{loading}/numerics.yaml",default_flow_style = False)

In [7]:
geometry = damask.Grid.load_ASCII(f'damask3/damask3_template/RVE_1_40_D.geom')

print(geometry)
print(geometry.material)
for loading in loading_paths:
    geometry.save(f"damask3/templates/{loading}/RVE_1_40_D.vti")

C:\Users\nguye\AppData\Local\Temp\ipykernel_8672\661526223.py:1: DeprecationWarning: Support for ASCII-based geom format will be removed in DAMASK 3.0.0
  geometry = damask.Grid.load_ASCII(f'damask3/damask3_template/RVE_1_40_D.geom')


cells:  40 × 40 × 40
size:   0.003 × 0.003 × 0.003 m³
origin: 0.0   0.0   0.0 m
# materials: 3165
[[[ 482  482  482 ... 1605 1605  162]
  [1121 1121 1121 ... 1605  162  162]
  [ 162  162 1121 ... 1605  162  162]
  ...
  [ 687 2575 2575 ... 2516 2516 2516]
  [2863 2863 2575 ... 2516 2516 2516]
  [2863 2863 1490 ... 2516 2516 2516]]

 [[ 482  482  482 ... 1605 1605  162]
  [ 162  482  482 ... 1605  162  162]
  [ 162  482  482 ... 1605  162  162]
  ...
  [ 687  687 2575 ... 2694 2694 2694]
  [1203 1203 1203 ... 2694 2694 2694]
  [1203 1203 1203 ... 2694 2694 2694]]

 [[ 162  482  482 ... 1605 1605  162]
  [ 162  482  482 ... 1605  162  162]
  [ 162  482  482 ... 1605  162  162]
  ...
  [ 687  506  506 ... 1164 2694 2349]
  [ 506  506 1768 ... 2694 2694 2349]
  [1203 1203 1768 ... 2233 1902 1902]]

 ...

 [[1285  761  761 ... 1698 2789 2789]
  [1285  761  761 ... 1698 1698 2789]
  [2985  761  761 ... 1698 1948 1948]
  ...
  [2221 2221 2575 ... 2773 2773 2177]
  [2789 1285 1285 ... 2773  28

In [11]:
def read_file_to_sentences(file_path):
    with open(file_path, "r") as file:
        content = file.read()
        sentences = content.split("\n")
        return sentences

def filteringSentences(sentences):
    indices = [2,4,6]
    filteredSentences = [sentence.split() for i, sentence in enumerate(sentences) if i % 2 != 0]
    filteredSentences = [[value for i, value in enumerate(sentence) if i in indices] for sentence in filteredSentences]
    return filteredSentences

sentences = read_file_to_sentences("damask3/damask3_template/PH_euler_angles.txt")
euler_angles = filteringSentences(sentences)


In [12]:
PH_loading_paths = [
    "PH_linear_uniaxial_RD",
    "PH_linear_uniaxial_TD",
    "PH_nonlinear_biaxial_RD",
    "PH_nonlinear_biaxial_TD",
    "PH_nonlinear_planestrain_RD",
    "PH_nonlinear_planestrain_TD",
    "PH_nonlinear_uniaxial_RD",
    "PH_nonlinear_uniaxial_TD",
]

material = damask.ConfigMaterial("""
homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}

phase:
  Aluminum:
    lattice: cF
    mechanical:
      output: [F, P, F_e, F_p, L_p, O]
      elastic: {type: Hooke, C_11: 107e9, C_12: 52e9, C_44: 26e9}
      plastic:
        type: phenopowerlaw
        output: [xi_sl, gamma_sl]       # xi_sl: resistance against plastic slip, gamma_sl: plastic shear
        N_sl: [12]                      # Nslip (DAMASK2)                 Number of slip systems (DAMASK3, DAMASK2 -)
        dot_gamma_0_sl: 1e-3            # gdot0_slip (DAMASK2)            Reference shear rate (DAMASK3, DAMASK2 s^-1)
        n_sl: 20                        # n_slip (DAMASK2)                Strain rate sensitivity (DAMASK3, DAMASK2 -)
        a_sl: 1.6                       # a_slip (DAMASK2)                Hardening exponent (DAMASK3, DAMASK2 -)
        h_0_sl-sl: 600e6                # h0_slipslip (DAMASK2)           Hardening slip value (DAMASK3, DAMASK2 Pa | Usually MPa => * e6)
        xi_0_sl: [55e6]                 # tau0_slip (DAMASK2)             Critical shear stress (DAMASK3, DAMASK2 Pa | Usually MPa => * e6)
        xi_inf_sl: [150e6]              # tausat_slip (DAMASK2)           Saturation shear stress (DAMASK3, DAMASK2 Pa | Usually MPa => * e6)
        h_sl-sl: [1, 1, 1, 1, 1, 1, 1]  # interaction_slipslip (DAMASK2)  Interaction coefficients (DAMASK3, DAMASK2 -)
        atol_xi: 1.0                    # atol resistance (DAMASK2)       

""").material_add(O=damask.Rotation.from_Euler_angles(np.array(euler_angles), degrees=True),
                  phase='Aluminum',
                  homogenization='SX')

#.material_add(O=damask.Rotation.from_random(shape=geometry.N_materials),
                  # phase='Al',
                  # homogenization='SX')
                  
for loading in PH_loading_paths:
    material.save(f"damask3/templates/{loading}/material.yaml")


In [14]:
def read_file_to_sentences(file_path):
    with open(file_path, "r") as file:
        content = file.read()
        sentences = content.split("\n")
        return sentences

def filteringSentences(sentences):
    indices = [2,4,6]
    filteredSentences = [sentence.split() for i, sentence in enumerate(sentences) if i % 2 != 0]
    filteredSentences = [[value for i, value in enumerate(sentence) if i in indices] for sentence in filteredSentences]
    return filteredSentences

sentences = read_file_to_sentences("damask3/damask3_template/DB_euler_angles.txt")
euler_angles = filteringSentences(sentences)


In [21]:
DB_loading_paths = [
    "DB_linear_uniaxial_RD",
    "DB_linear_uniaxial_TD",
    "DB_nonlinear_biaxial_RD",
    "DB_nonlinear_biaxial_TD",
    "DB_nonlinear_planestrain_RD",
    "DB_nonlinear_planestrain_TD",
    "DB_nonlinear_uniaxial_RD",
    "DB_nonlinear_uniaxial_TD",
]

material = damask.ConfigMaterial("""
homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}

phase:
  Aluminum:
    lattice: cF
    mechanical:
      output: [F, P, F_e, F_p, L_p, O]
      elastic: {type: Hooke, C_11: 107e9, C_12: 52e9, C_44: 26e9}
      plastic:
        type: dislotwin
        output: [rho_dip, rho_mob]
        D: 15.25e-6                      # grainsize (DAMASK2)               Average grain size (DAMASK3, DAMASK2 m | Usually μm => * e-6)
        tau_0: [55e6]                   # SolidSolutionStrength (DAMASK2)   Solid solution strength (DAMASK3, DAMASK2 Pa | Usually MPa => * e6)
        N_sl: [12]                      # Nslip (DAMASK2)                   Number of slip systems (DAMASK3, DAMASK2 -)            
        b_sl: [0.2546e-9]               # slipburgers (DAMASK2)             Burgers vector magnitude (DAMASK3, DAMASK2 m | Usually nm => * e-9)
        rho_dip_0: [1]                  # rhoedgedip0 (DAMASK2)             Dislocation dipole density (DAMASK3, DAMASK2 m^-2)
        rho_mob_0: [1e12]               # rhoedge0 (DAMASK2)                Dislocation mobile density (DAMASK3, DAMASK2 m^-2)
        v_0: [10e-5]                    # v0 (DAMASK2)                      Dislocation velocity prefactor (DAMASK3, DAMASK2 m·s^-1) 
        Q_sl: [0.8361e-19]              # Qedge (DAMASK2) or Delta_F        Activation energy for dislocation glide (DAMASK3, DAMASK2 J) 
        Q_cl: 1.876e-19                 # Q_sd (DAMASK2)                    Activation energy for climb (DAMASK3, DAMASK2 J) 
        p_sl: [0.8]                     # p_slip (DAMASK2)                  p-exponent in glide velocity (DAMASK3, DAMASK2 -)  
        q_sl: [1.5]                     # q_slip (DAMASK2)                  q-exponent in glide velocity (DAMASK3, DAMASK2 -)  
        i_sl: [45]                      # CLambdaSlip (DAMASK2)             Dislocation mean free path (DAMASK3, DAMASK2 -)  
        V_at: 0.066e-27                 # Catomicvolume (DAMASK2)           Atomic volume (DAMASK3 m^3 | DAMASK2 b^3 | Usually nm^3 => * e-27)
        D_a: 7                          # Cedgedipmindistance (DAMASK2)     Minimum dipole distance (DAMASK3, DAMASK2 b)
        D_0: 6.23e-4                    # D0 (DAMASK2)                      Vacancy diffusion prefactor (DAMASK3, DAMASK2 m^2·s^-1)  
        h_sl-sl: [1, 1, 1, 1, 1, 1, 1]  # interaction_slipslip (DAMASK2)    Interaction coefficients (DAMASK3, DAMASK2 -)  

""").material_add(O=damask.Rotation.from_Euler_angles(np.array(euler_angles), degrees=True),
                  phase='Aluminum',
                  homogenization='SX')
    # .material_add(O=damask.Rotation.from_random(shape=geometry.N_materials),
    #               phase='Al',
    #               homogenization='SX')
                  
for loading in DB_loading_paths:
    material.save(f"damask3/templates/{loading}/material.yaml")


In [ ]:
DB_loading_paths = [
    "DB_linear_uniaxial_RD",
    "DB_linear_uniaxial_TD",
    "DB_nonlinear_biaxial_RD",
    "DB_nonlinear_biaxial_TD",
    "DB_nonlinear_planestrain_RD",
    "DB_nonlinear_planestrain_TD",
    "DB_nonlinear_uniaxial_RD",
    "DB_nonlinear_uniaxial_TD",
]

material = damask.ConfigMaterial("""
homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}

phase:
  Aluminum:
    lattice: cF
    mechanical:
      output: [F, P, F_e, F_p, L_p, O]
      elastic: {type: Hooke, C_11: 107e9, C_12: 52e9, C_44: 26e9}
      plastic:
        type: dislotwin
        output: [rho_dip, rho_mob, gamma_sl, Lambda_sl, tau_pass, f_tw, Lambda_tw, f_tr]
        output: [rho_u_ed_pos, rho_b_ed_pos, rho_u_ed_neg, rho_b_ed_neg, rho_u_sc_pos, rho_b_sc_pos, rho_u_sc_neg, rho_b_sc_neg, rho_d_ed, rho_d_sc]
        B: [0.001]                      # Drag coefficient                  Drag coefficient (DAMASK3, DAMASK2, Usually Pa·s)
        grainsize:                15.25e-6        
        SolidSolutionStrength:    55e6      
        Nslip:                    12
        slipburgers:              25.46e-11        
        rhoedgedip0:              1.0            
        rhoedge0:                 1.0e12         
        v0:                       10e-5          
        Qedge:                    0.8361e-19    
        p_slip:                   0.8          
        q_slip:                   1.5       
        CLambdaSlip:              45          
        D0:                       6.23e-4     
        Qsd:                      1.876e-19    
        Catomicvolume:            4.0            
        Cedgedipmindistance:      7.0             
        atol_rho:                 1.0
        interaction_slipslip:     1 1 1 1 1 1  

        B: [0.001]                      # Drag coefficient                  Drag coefficient (DAMASK3, DAMASK2, Usually Pa·s)
        D: 5.25e-6                      # grainsize (DAMASK2)               Average grain size (DAMASK3, DAMASK2 m | Usually μm => * e-6)
        tau_0: [55e6]                   # SolidSolutionStrength (DAMASK2)   Solid solution strength (DAMASK3, DAMASK2 Pa | Usually MPa => * e6)
        N_sl: [12]                      # Nslip (DAMASK2)                   Number of slip systems (DAMASK3, DAMASK2 -)            
        b_sl: [0.2546e-9]               # slipburgers (DAMASK2)             Burgers vector magnitude (DAMASK3, DAMASK2 m | Usually nm => * e-9)
        rho_dip_0: [1]                  # rhoedgedip0 (DAMASK2)             Dislocation dipole density (DAMASK3, DAMASK2 m^-2)
        rho_mob_0: [1e12]               # rhoedge0 (DAMASK2)                Dislocation mobile density (DAMASK3, DAMASK2 m^-2)
        v_0: [10e-5]                    # v0 (DAMASK2)                      Dislocation velocity prefactor (DAMASK3, DAMASK2 m·s^-1) 
        Q_sl: [0.8361e-19]              # Qedge (DAMASK2) or Delta_F        Activation energy for slip (DAMASK3, DAMASK2 J) 
        Q_cl: 1.876e-19                 # Q_sd (DAMASK2)                    Activation energy for climb (DAMASK3, DAMASK2 J) 
        p_sl: [0.8]                     # p_slip (DAMASK2)                  p-exponent in glide velocity (DAMASK3, DAMASK2 -)  
        q_sl: [1.5]                     # q_slip (DAMASK2)                  q-exponent in glide velocity (DAMASK3, DAMASK2 -)  
        i_sl: [45]                      # CLambdaSlip (DAMASK2)             Dislocation mean free path (DAMASK3, DAMASK2 -)  
        V_at: 0.066e-27                 # Catomicvolume (DAMASK2)           Atomic volume (DAMASK3 m^3 | DAMASK2 b^3 | Usually nm^3 => * e-27)
        D_a: 7                          # Cedgedipmindistance (DAMASK2)     Minimum dipole distance (DAMASK3, DAMASK2 b)
        D_0: 6.23e-4                    # D0 (DAMASK2)                      Vacancy diffusion prefactor (DAMASK3, DAMASK2 m^2·s^-1)  
        h_sl-sl: [1, 1, 1, 1, 1, 1, 1]  # interaction_slipslip (DAMASK2)    Interaction coefficients (DAMASK3, DAMASK2 -)  


""").material_add(O=damask.Rotation.from_Euler_angles(np.array(euler_angles), degrees=True),
                  phase='Aluminum',
                  homogenization='SX')
    # .material_add(O=damask.Rotation.from_random(shape=geometry.N_materials),
    #               phase='Al',
    #               homogenization='SX')
                  
for loading in DB_loading_paths:
    material.save(f"damask3/templates/{loading}/material.yaml")
